In [ ]:
from pathlib import Path
import time
import datetime

import chainer
import chainer.links as L
import chainer.functions as F
from chainer.datasets import TransformDataset
from chainercv import transforms
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
import cupy as cp
import pandas as pd
from PIL import Image, ImageOps

In [ ]:
class Dataset(chainer.dataset.DatasetMixin):
    def __init__(self, df, data, train=True, transform=None):
        self.df = pd.read_csv(df)
        self.data = data.transpose(0, 3, 1, 2).astype(np.float32)
        self.transforms = transforms
        self.train = train
        if self.train:
            self.y = self.df["target"].values

    def __len__(self):
        return len(self.df)
            
    def get_example(self, index):
        row = self.df.iloc[index]
        x = self.data[index]

        if self.transform:
            x = self.transform(x)
        
        if self.train:
            y = row["target"]
            return x, y
        else:
            return x


class Net(chainer.Chain):
    def __init__(self, arch):
        super(Net, self).__init__()
        with self.init_scope():
            self.arch = arch
            self.arch.fc6 = L.Linear(None, 1)
            
    def forward(self, x):
        x = self.arch.extract(x)["pool5"]
        x = self.arch.fc6(x)
        return x


class Loss():
    
    def __init__(self, model, gamma=2.0):
        self.model = model
        self.gamma = gamma

    def __call__(self, *args, **kwargs):
        x = self.model.forward(*args[:-1])
        loss = self._loss_func(x, args[-1])
        return loss
    
    def _loss_func_focal(self, x, t):
        ce_loss = F.sigmoid_cross_entropy(x, t[..., None], reduce='no')
        pt = F.exp(-ce_loss)
        self.loss = F.mean((1 - pt)**self.gamma * ce_loss)
        self.accuracy = F.binary_accuracy(x.array.flatten(), t)
        return self.loss

    def _loss_func(self, x, t):
        self.loss = F.sigmoid_cross_entropy(x, t[..., None], reduce='mean')
        self.accuracy = F.binary_accuracy(x.array.flatten(), t)
        return self.loss

In [ ]:
#train = Dataset(filename=Path.home() / "dataset/Melanoma/train.csv")
train_ds = Dataset(df=Path.home() / "dataset/Melanoma/train.csv",
                   data=np.load("x_train_224.npy"))
test_ds = Dataset(df=Path.home() / "dataset/Melanoma/test.csv",
                  data=np.load("x_test_224.npy"))

In [ ]:
train_ds = Dataset(train_df)
train_itr = chainer.iterators.MultiprocessIterator(train_ds, 30, repeat=False, shuffle=True)

In [ ]:
train_itr.reset()
batch = next(train_itr)

In [ ]:
epochs = 10
batch_size = 30
device = 0
title = "test"
output_dir = Path("../../results") / title
output_dir.mkdir(parents=True, exist_ok=True)
es_patience = 3  # Early Stopping patience
skf = StratifiedKFold(n_splits=5, random_state=47, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(skf.split(X=train_df, y=train_df.target), 1):
    print('=' * 20, 'Fold', fold, '=' * 20)
    best_val = None
    patience = es_patience
    
    train_ds = Dataset(train_df.iloc[train_idx])
    val_ds = Dataset(train_df.iloc[val_idx])
    train_itr = chainer.iterators.MultiprocessIterator(train_ds, batch_size, repeat=False, shuffle=True)
    val_itr = chainer.iterators.MultiprocessIterator(val_ds, batch_size, repeat=False, shuffle=False)
    
    arch = chainer.links.ResNet50Layers()
    model = Net(arch)
    if device >= 0:
        model.to_gpu(device)
    
    optimizer = chainer.optimizers.Adam(alpha=0.001)
    optimizer.setup(model)
    loss_func = Loss(model)
    
    for epoch in range(epochs):
        start_time = time.time()
        epoch_loss = 0
        train_itr.reset()
        val_itr.reset()
        
        with chainer.using_config("train", True):
            for batch in train_itr:
                x = [img for img, _ in batch]
                t = chainer.dataset.to_device(device, np.array([label for _, label in batch]))
                optimizer.update(loss_func, x, t)
                epoch_loss += loss_func.loss.item()
                print("\r{} / {} : [loss = {:.3f}, acc = {:.3f}]".format(
                    train_itr.current_position, train_itr._epoch_size, 
                    loss_func.loss.item(), loss_func.accuracy.item()), end="")
            print("finish training")
            
        with chainer.using_config("train", False):
            val_preds = np.zeros(len(val_ds))
            for idx, batch in enumerate(val_itr):
                x = [img for img, _ in batch]
                t = chainer.dataset.to_device(device, np.array([label for _, label in batch]))
                pred = cp.asnumpy(F.sigmoid(model(x)).array).flatten()
                val_preds[idx * len(batch): (idx + 1) * len(batch)] = pred
            val_accuracy = accuracy_score(val_ds.y, np.round(val_preds))
            val_roc = roc_auc_score(val_ds.y, val_preds)
            
            print('Epoch {:03}: | Train Loss: {:.3f} | Val acc: {:.3f} | Val roc_auc: {:.3f} | time : {}'.format(
                epoch + 1, epoch_loss, val_accuracy, val_roc, str(datetime.timedelta(seconds=time.time() - start_time))))

            if not best_val:
                best_val = val_roc
                chainer.serializers.save_npz(str(output_dir / f"roc_{fold:02}.npz"), model)
            elif val_roc > best_val - 1.e-5:
                best_val = val_roc
                patience = es_patience
                chainer.serializers.save_npz(str(output_dir / f"roc_{fold:02}.npz"), model)
            else:
                patience -= 1
                if patience == 0:
                    print("Early stopping!")
                    continue
    chainer.serializers.save_npz(str(output_dir / f"epoch_{fold:02}.npz"), model)
    print("Finish Fold{}. Best Val roc_aud : {:.3f}".format(fold, best_val))

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X=train_df, y=train_df.target), 1):
    test_ds = Dataset(test_df, train=False)
    test_itr = chainer.iterators.MultiprocessIterator(test_ds, batch_size, shuffle=False, repeat=False)
    
    arch = chainer.links.ResNet50Layers()
    model = Net(arch)
    chainer.serializers.load_npz(str(output_dir / f"roc_{fold:02}.npz"), model)
    
    preds = np.zeros(len(test_ds))
    for idx, batch in enumerate(test_itr):
        batch = chainer.datasets.concat_example(batch, device)
        pred = F.sigmoid(model.forward(batch))
        preds[idx * batch_size:(idx + 1) * batch_size] += pred / skf.n_splits